In [1]:
import json 
import pandas as pd
import matplotlib.pyplot as plt

### Data

In [2]:
match = pd.read_csv('Data/matches.csv')

In [3]:
match.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode
0,2015-03-04,AFG,AUS,Away,3773,65
1,2012-08-25,AFG,AUS,Neutral,3437,156
2,2019-06-01,AFG,AUS,Neutral,4306,149
3,2014-03-01,AFG,BAN,Away,3623,190
4,2016-09-25,AFG,BAN,Away,3936,198


In [4]:
attr = pd.read_csv("Data/attributes.csv")

In [5]:
attr.head()

,Full Name,Born,Bats,Bowls,Code
0,David A Warner,1986-10-27,Left Handed,Right Arm Leg Break,3637
1,Aaron James Finch,1986-11-17,Right Handed,Left Arm Orthodox,3845
2,Steven Peter Devereux Smith,1989-02-06,Right Handed,Right Arm Leg Break,3756
3,Glenn James Maxwell,1988-10-14,Right Handed,Right Arm Off Break,3969
4,James Peter Faulkner,1990-04-29,Right Handed,Left Arm Medium,3930


In [6]:
f = open('Data/scorecard.json',) 
scores = json.load(f)

In [7]:
len(scores)

3649

In [8]:
f = open('Data/fielders1.json',) 
fielders = json.load(f)

In [9]:
len(fielders)

3648

In [10]:
fielders_keys = fielders.keys()

In [11]:
matches_fielders=pd.read_hdf('Data/processed_data.h5')

In [12]:
matches_fielders.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,Winner
0,2015-03-04,AFG,AUS,1,3773,65,1,142,417,AUS
1,2012-08-25,AFG,AUS,2,3437,156,2,206,272,AUS
2,2019-06-01,AFG,AUS,2,4306,149,1,207,209,AUS
3,2014-03-01,AFG,BAN,1,3623,190,2,254,222,AFG
4,2016-09-25,AFG,BAN,1,3936,198,2,258,265,BAN


In [13]:
f = open('Data/tmap.json',) 
tmap = json.load(f)

### Adding fielders

In [14]:
matches_fielders=matches_fielders.drop(['Date','Team_1','Team_2','Winner'],axis=1)

In [15]:
len(matches_fielders)

3656

In [16]:
matches_fielders['catches_team1']=""
matches_fielders['catches_team2']=""
matches_fielders['stumped_team1']=""
matches_fielders['stumped_team2']=""

In [17]:
matches_fielders.head()

,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,catches_team1,catches_team2,stumped_team1,stumped_team2
0,1,3773,65,1,142,417,,,,
1,2,3437,156,2,206,272,,,,
2,2,4306,149,1,207,209,,,,
3,1,3623,190,2,254,222,,,,
4,1,3936,198,2,258,265,,,,


In [18]:
max_len=0
temp=0
index=""

for i in fielders_keys:
    c1=0
    c2=0
    st1=0
    st2=0
    for j in range(0,len(fielders[i])):
        if('c1' in fielders[i][j][1].keys()):
            c1=c1+1
        elif('c2' in fielders[i][j][1].keys()):
            c2=c2+1
        elif('st1' in fielders[i][j][1].keys()):
            st1=st1+1
        elif('st2' in fielders[i][j][1].keys()):
            st2=st2+1
        temp=max(st1,st2,c1,c2)
    if temp>=max_len:
        index=i
        max_len=temp


In [19]:
print(max_len)
print(index)

10
3757


In [20]:
na=[]

In [21]:
for k in range(0,len(matches_fielders)):
    
    match_code = str(matches_fielders['MatchCode'].iloc[k])
    
    if(match_code not in fielders_keys):
        na.append(k)
        continue
    
    catch_team1=[0]*10
    catch_team2=[0]*10
    stump_team1=[0]*10
    stump_team2=[0]*10
    
    for j in range(0,len(fielders[match_code])):
        
        if('c1' in fielders[match_code][j][1].keys()):
            catch_team1[fielders[match_code][j][0]]=1
        elif('c2' in fielders[match_code][j][1].keys()):
            catch_team2[fielders[match_code][j][0]]=1
        elif('st1' in fielders[match_code][j][1].keys()):
            stump_team1[fielders[match_code][j][0]]=1
        elif('st2' in fielders[match_code][j][1].keys()):
            stump_team2[fielders[match_code][j][0]]=1
            
    
     
    matches_fielders['catches_team1'].iloc[k]=catch_team1
    matches_fielders['catches_team2'].iloc[k]=catch_team2
    matches_fielders['stumped_team1'].iloc[k]=stump_team1
    matches_fielders['stumped_team2'].iloc[k]=stump_team2


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [22]:
matches_fielders

,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,catches_team1,catches_team2,stumped_team1,stumped_team2
0,1,3773,65,1,142,417,"[1, 1, 1, 1, 1, 1, 0, 1, 0, 1]","[1, 1, 1, 1, 0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,2,3437,156,2,206,272,"[0, 0, 1, 1, 0, 0, 0, 1, 1, 1]","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
2,2,4306,149,1,207,209,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 0, 0, 1, 1, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
3,1,3623,190,2,254,222,"[0, 0, 0, 0, 1, 1, 0, 0, 1, 1]","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,1,3936,198,2,258,265,"[0, 1, 0, 1, 1, 1, 1, 0, 0, 1]","[1, 1, 0, 1, 1, 0, 0, 1, 1, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...
3651,2,1765,58,1,91,138,,,,
3652,2,1770,65,1,178,134,,,,
3653,2,1862,56,2,173,175,,,,
3654,2,1866,2,1,155,154,,,,


In [23]:
na

[546, 3116, 3650, 3651, 3652, 3653, 3654, 3655]

In [24]:
matches_fielders.drop(matches_fielders.index[na])

,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,catches_team1,catches_team2,stumped_team1,stumped_team2
0,1,3773,65,1,142,417,"[1, 1, 1, 1, 1, 1, 0, 1, 0, 1]","[1, 1, 1, 1, 0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,2,3437,156,2,206,272,"[0, 0, 1, 1, 0, 0, 0, 1, 1, 1]","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
2,2,4306,149,1,207,209,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 0, 0, 1, 1, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
3,1,3623,190,2,254,222,"[0, 0, 0, 0, 1, 1, 0, 0, 1, 1]","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,1,3936,198,2,258,265,"[0, 1, 0, 1, 1, 1, 1, 0, 0, 1]","[1, 1, 0, 1, 1, 0, 0, 1, 1, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...
3645,2,915,36,1,233,99,"[1, 1, 0, 1, 0, 1, 0, 1, 0, 0]","[1, 0, 0, 1, 1, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3646,2,1115,36,2,155,151,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 1, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
3647,2,1575,128,2,247,244,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 0, 1, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3648,2,1703,160,1,186,256,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [25]:
#matches_fielders.to_csv('matches_fielders.csv')